In [1]:
import pickle
import dynet
import os
import numpy as np
%run utils.ipynb

In [2]:

if __name__ == '__main__':
    (vocab, net_properties) = pickle.load(open('../data/vocab_net.data', 'rb'))
    network = Network(vocab, net_properties)
    network.load('../data/net.model')

    writer = open('../data/predictions.data', 'w')
    feature_list = pickle.load(open( "../data/test.data", "rb" ))

    correct = 0
    shiftstars, tps, fps, fns, shifts, tpss, fpss, fnss = 0, 0, 0, 0, 0, 0, 0, 0
    for feature_set in feature_list:
        pred = network.decode(feature_set[:-1])
        writer.write("ground truth: " + feature_set[-1] + ", prediction: " + pred + "\n\n")

        # count shift stars
        if pred == "shift star":
            if feature_set[-1] == "shift star":
                shiftstars += 1
                tpss += 1
            else:
                fpss += 1
        if feature_set[-1] == "shift star" and pred != "shift star":
            fnss += 1
            shiftstars += 1

        # count shifts
        if pred == "shift":
            if feature_set[-1] == "shift":
                shifts += 1
                tps += 1
            else:
                fps += 1
        if feature_set[-1] == "shift" and pred != "shift":
            fns += 1
            shifts += 1

        # total accuracy
        if pred == feature_set[-1]:
            correct += 1


    print("accuracy: " + str(float(correct)/len(feature_list)))

    print("----------------------------------------")

    precision_ss = tpss / (tpss + fpss)
    recall_ss = tpss / (tpss + fnss)
    print("star precision: " + str(precision_ss))
    print("star recall: " + str(recall_ss))
    print("total stars: " + str(shiftstars))
    print("star F1: " + str( 2*(precision_ss * recall_ss) / (precision_ss + recall_ss) ))

    print("----------------------------------------")

    precision_s = tps / (tps + fps)
    recall_s = tps / (tps + fns)
    print("shift precision: " + str(precision_s))
    print("shift recall: " + str(recall_s))
    print("total shifts: " + str(shifts))
    print("shift F1: " + str( 2*(precision_s * recall_s) / (precision_s + recall_s) ))

    writer.close()

accuracy: 0.8461
----------------------------------------
star precision: 0.8074074074074075
star recall: 0.6728395061728395
total stars: 162
star F1: 0.7340067340067341
----------------------------------------
shift precision: 0.8660848630755341
shift recall: 0.9392950391644909
total shifts: 3064
shift F1: 0.9012055738218255


In [3]:
def remove_star(s):
    s = s.split()
    s = list(filter(lambda x: '*' not in x, s))
    return ' '.join(s)

def action(b, s, p):
    error = None

    if p.split()[0] == 'shift':
        if len(p.split()) > 1 and p.split()[1] == 'star':
            s.append(Node('*'))
            
        # normal shift
        try: s.append(b.pop(0))
        except: error = 'pop on empty buffer'

    elif p.split()[0] == 'unary':
        n = Node(clean(p.split()[1]))
        try:
            n.l = s.pop()
            s.append(n)
        except: error = 'unary on empty stack'

    else: # p.split()[0] == 'binary':
        n = Node(clean(p.split()[1]))
        try:
            n.r, n.l = s.pop(), s.pop()
            s.append(n)
        except: error = 'binary on insufficient stack'

    return b, s, error


if __name__ == '__main__':
    # load the network
    (vocab, net_properties) = pickle.load(open('../data/vocab_net.data', 'rb'))
    network = Network(vocab, net_properties)
    network.load('../data/net.model')

    # open treebank for testing
    treepath = "../treebank/treebank_3/parsed/mrg/wsj/23"

    # open file and save as one large string
    text = ""
    for filename in os.listdir(treepath):
        if filename.startswith('.'):
            continue
        with open(treepath + "/" + filename, 'r') as f:
            text += f.read().replace('\n', '')

    tree_string_list = []
    s = []
    start = 0
    for i in range(len(text)):
        if text[i] == "(":
            s.append("(")
        elif text[i] == ")":
            s.pop()
            if not s:
                tree_string_list.append(text[start : i + 1])
                start = i + 1

    # turn tree strings into tree_list
    tree_list = []
    for t in tree_string_list:
        tree_list.append((parse_tree(t[1:-1])))

    # use inorder traveral to generate sentences from trees
    sentences = []
    for t in tree_list:
        sentences.append(remove_star(inorder_sentence(t).lstrip()))

    # testing

    with open('../data/tree_pred.txt', 'w') as outfile:
        for s, t in zip(sentences, tree_list):
            s = [clean(x) for x in s.split()]
            
            #debug
            print(' '.join(s) + '\n') # print sentence
            outfile.write(' '.join(s) + '\n\n')

            # construct tree
            buff = list(map(Node, s))
            stack = []
            infinite_loop_count = 0 # terminate after 100 moves
            printed_from_error = False
            while buff or len(stack) > 1: # end when buff consumed & stack has tree
                
                
                # cast to string and predict
                stack, buff = list(map(tree_to_str, stack)), list(map(tree_to_str, buff))
                try: f = extract_features(datum(stack, buff, None))
                except: 
                    print('feature extraction error')
                    printed_from_error = True
                    break
                    

                pred = network.decode(rearrange([0] + f)[:-1])
                # outfile.write(str(f) + ' ' +  pred + '\n')

                # cast back to Node and complete action
                stack, buff = list(map(Node, stack)), list(map(Node, buff))
                buff, stack, error = action(buff, stack, pred)
                if error:
                    # outfile.write(error + '\n')
                    print('Error: ' + error)
                    print(stack_to_str(stack) + '\n')
                    printed_from_error = True
                    outfile.write('Error: '+ error + '\n')
                    break
#                 print(pred + '\n' + stack_to_str(stack) + '\n')
                outfile.write(pred + '\n' + stack_to_str(stack) + '\n\n')
                
                infinite_loop_count += 1
                if infinite_loop_count >= 150: 
                    print('infinite loop error')
                    print(stack_to_str(stack) + '\n')
                    printed_from_error = True
                    break
                
            if not printed_from_error: print(stack_to_str(stack) + '\n')
            print('GROUND TRUTH:\n' + tree_to_str(t) + '\n')
            print('-------------------end of sentence-----------------\n')
            outfile.write('GROUND TRUTH:\n' + tree_to_str(t) + '\n\n')
            outfile.write('-------------------end of sentence-----------------\n\n')



No , it was n't Black Monday .

[ (S (NNS No) (S (, ,) (S (NP (PRP it)) (S (VP (VBD was) (NP (RB n't) (NP (NNP Black) (NNP Monday)))) (. .)))))]

GROUND TRUTH:
 (S (INTJ (RB No)) (S_inner (, ,) (S_inner (NP-SBJ (PRP it)) (S_inner (VP (VBD was) (VP_inner (RB n't) (NP-PRD (NNP Black) (NNP Monday)))) (. .)))))

-------------------end of sentence-----------------

But while the New York Stock Exchange did n't fall apart Friday as the Dow Jones Industrial Average plunged 190.58 points -- most of it in the final hour -- it barely managed to stay this side of chaos .

[ (S (CC But) (S (IN while) (S (NP (DT the) (NP (NNP New) (NP (NNP York) (NP (NNP Stock) (NNP Exchange))))) (S (VBD did) (S (NP (RB n't) (NP (NN fall) (NNS apart))) (S (NNP Friday) (S (IN as) (S (NP (DT the) (NP (NNP Dow) (NP (NNP Jones) (NP (NNP Industrial) (NNP Average))))) (S (VBZ plunged) (S (NP (CD 190.58) (NNS points)) (S (: --) (S (NP (NP (NNS most)) (PP (IN of) (NP (NP (PRP it)) (PP (IN in) (NP (DT the) (NP (JJ final) (N


GROUND TRUTH:
 (SINV (S-TPC-2 (ADVP-TMP (RB Once) (RB again)) (S-TPC-2_inner (-LRB- -LCB-) (S-TPC-2_inner (NP-SBJ-3 (DT the) (NNS specialists)) (S-TPC-2_inner (-RRB- -RCB-) (VP (VBD were) (VP_inner (RB not) (ADJP-PRD (JJ able) (S (NP-SBJ (-NONE- *-3)) (VP (TO to) (VP (VB handle) (NP (NP (DT the) (NNS imbalances)) (PP-LOC (IN on) (NP (NP (DT the) (NN floor)) (PP (IN of) (NP (DT the) (NP_inner (NNP New) (NP_inner (NNP York) (NP_inner (NNP Stock) (NNP Exchange))))))))))))))))))) (SINV_inner (, ,) (SINV_inner ('' '') (SINV_inner (VP (VBD said) (S (-NONE- *T*-2))) (SINV_inner (NP-SBJ (NP (NNP Christopher) (NNP Pedersen)) (NP-SBJ_inner (, ,) (NP (NP (JJ senior) (NP_inner (NN vice) (NN president))) (PP-LOC (IN at) (NP (NNP Twenty-First) (NP_inner (NNP Securities) (NNP Corp))))))) (. .))))))

-------------------end of sentence-----------------

Countered 0 James Maguire , chairman of specialists Henderson Brothers Inc. : `` It is easy to say 0 the specialist is n't doing his job .

[ (S (VBD 

[ (S (PP (IN In) (NP (NP (NN percentage) (NNS terms)) (NP (, ,) (NP (NP (RB however)) (SBAR (, ,) (S (NP (NP (DT the) (NP (NNP Dow) (POS 's))) (NN dive)) (VP (VBD was) (NP (NP (DT the) (NP (NN 12th-worst) (NNS ever))) (NP (CC and) (NP (NP (DT the) (NN sharpest)) (PP (IN since) (S (NP (DT the) (NN market)) (VP (VBZ fell) (NP (JJ 156.83) (NP (, ,) (NP (CC or) (NP (NP (CD 8) (NN %)) (NP (, ,) (NP (NP (DT a) (NN week)) (PP (IN after) (NP (NNP Black) (NNP Monday)))))))))))))))))))))) (. .))]

GROUND TRUTH:
 (S (PP-LOC (IN In) (NP (NN percentage) (NNS terms))) (S_inner (, ,) (S_inner (ADVP (RB however)) (S_inner (, ,) (S_inner (NP-SBJ (NP (DT the) (NP_inner (NNP Dow) (POS 's))) (NN dive)) (S_inner (VP (VBD was) (NP-PRD (NP (NP (DT the) (JJ 12th-worst)) (ADVP-TMP (RB ever))) (NP-PRD_inner (CC and) (NP (NP (DT the) (JJS sharpest)) (SBAR-TMP (IN since) (S (NP-SBJ (DT the) (NN market)) (VP (VBD fell) (VP_inner (NP-EXT (NP (CD 156.83)) (NP-EXT_inner (, ,) (NP-EXT_inner (CC or) (NP (CD 8) (NN %)))

infinite loop error
[ (PP (IN For) (NP (NN example))),  (, ,),  (PRP$ their),  (VBG selling),  (NP (JJ caused) (NP (NN trading) (NNS halts))),  (TO to),  (VB be),  (NP (NP (NN declared)) (PP (IN in) (NP (NNP USAir) (NNP Group)))),  (, ,),  (WHNP (WDT which)),  *,  (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (NP (TO closed))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))]

GROUND TRUTH:
 (S (PP (IN For) (NP (NN example))) (S_inner (, ,) (S_inner (NP-SBJ (PRP$ their) (NN selling)) (S_inner (VP (VBD caused) (S (NP-SBJ-5 (NP (NN trading) (NNS halts)) (PP-LOC 


GROUND TRUTH:
 (S (PP (IN Without) (NP (NP (DT the) (NN guidepost)) (PP (IN of) (NP (NP (JJ stock-index) (NNS futures)) (NP_inner (: --) (NP_inner (NP (NP (DT the) (NN barometer)) (PP (IN of) (SBAR-NOM (WHADVP-2 (WRB where)) (S (NP-SBJ (NNS traders)) (VP (VBP think) (SBAR (-NONE- 0) (S (NP-SBJ-1 (DT the) (NP-SBJ-1_inner (JJ overall) (NP-SBJ-1_inner (NN stock) (NN market)))) (VP (VBZ is) (VP (VBN headed) (VP_inner (NP (-NONE- *-1)) (ADVP-DIR (-NONE- *T*-2)))))))))))) (: --))))))) (S_inner (NP-SBJ-3 (JJ many) (NNS traders)) (S_inner (VP (VBD were) (ADJP-PRD (JJ afraid) (S (NP-SBJ (-NONE- *-3)) (VP (TO to) (VP (VB trust) (NP (NP (NN stock) (NNS prices)) (VP (VBN quoted) (VP_inner (NP (-NONE- *)) (PP-LOC (IN on) (NP (DT the) (NP_inner (NNP Big) (NNP Board)))))))))))) (. .))))

-------------------end of sentence-----------------

The futures halt was even assailed by Big Board floor traders .

[ (S (NP (DT The) (NP (NNP futures) (NN halt))) (S (VP (VBD was) (VP (ADVP (RB even)) (VP (NP (NO

...

[ (S (`` ``) (S (S (NP (NNS These)) (VP (VBP are) (NP (RB n't) (NP (JJ mature) (NNS assets))))) (S (, ,) (S (CC but) (S (NP (PRP they)) (S (PP (IN have) (NP (DT the) (S (JJ potential) (S (NP (NONE 0)) (VP (TO to) (VP (VB be) (NP (RB so)))))))) (S (, ,) (S (NP (NNS '')) (S (VP (VBD said) (NP (NNP Mr.) (NNS Shidler))) (. .))))))))))]

GROUND TRUTH:
 (SINV (`` ``) (SINV_inner (S-TPC-2 (S (NP-SBJ (DT These)) (VP (VBP are) (VP_inner (RB n't) (NP-PRD (JJ mature) (NNS assets))))) (S-TPC-2_inner (, ,) (S-TPC-2_inner (CC but) (S (NP-SBJ (PRP they)) (VP (VBP have) (NP (NP (DT the) (NN potential)) (SBAR (WHADVP-1 (-NONE- 0)) (S (NP-SBJ (-NONE- *)) (VP (TO to) (VP (VB be) (VP_inner (ADVP-PRD (RB so)) (ADVP (-NONE- *T*-1))))))))))))) (SINV_inner (, ,) (SINV_inner ('' '') (SINV_inner (VP (VBD said) (S (-NONE- *T*-2))) (SINV_inner (NP-SBJ (NNP Mr.) (NNP Shidler)) (. .)))))))

-------------------end of sentence-----------------

`` Managed properly , and with a long-term outlook , these can become inv

[ (S (NP (NNP Stock) (NN fund)) (S (VP (VBD redemptions) (SBAR (IN during) (S (NP (DT the) (NP (CD 1987) (NN debacle))) (VP (VBD did) (VP (NP (RB n't) (NP (NN begin) (PP (TO to) (NP (NN snowball) (NNS until))))) (PP (IN after) (NP (NP (DT the) (NN market)) (PP (IN opened) (PP (IN on) (NP (NN Black) (NNP Monday))))))))))) (. .)))]

GROUND TRUTH:
 (S (NP-SBJ-1 (NN Stock) (NP-SBJ-1_inner (NN fund) (NNS redemptions))) (S_inner (PP-TMP (IN during) (NP (DT the) (NP_inner (CD 1987) (NN debacle)))) (S_inner (VP (VBD did) (VP_inner (RB n't) (VP (VB begin) (VP_inner (S (NP-SBJ (-NONE- *-1)) (VP (TO to) (VP (VB snowball)))) (PP-TMP (IN until) (SBAR (IN after) (S (NP-SBJ (DT the) (NN market)) (VP (VBD opened) (PP-TMP (IN on) (NP (NNP Black) (NNP Monday))))))))))) (. .))))

-------------------end of sentence-----------------

But fund managers say 0 they 're ready .

[ (S (CC But) (S (NP (NN fund) (NNS managers)) (S (VP (VBP say) (PP (NONE 0) (NP (NP (PRP they)) (NP (NN 're) (NNS ready))))) (. .)))

[ (S (NP (CC And) (SBAR (JJ today) (S (NP (PRP he)) (VP (MD 'll) (VP (VB be) (S (NP (NONE looking)) (VP (TO to) (VP (VB buy) (VP (NP (NN drug) (NNS stocks)) (VP (PP (IN like) (NP (NNP Eli) (NNP Lilly))) (VP (, ,) (VP (NNP Pfizer) (VP (CC and) (VP (NP (NNP American) (NP (NNP Home) (NNP Products))) (VP (NP (NN whose) (NP (NN dividend) (NNS yields))) (VP (IN have) (VP (VBN been) (VP (NNP bolstered) (PP (IN by) (NP (NN stock) (NN declines))))))))))))))))))))) (. .))]

GROUND TRUTH:
 (S (CC And) (S_inner (NP-TMP (NN today)) (S_inner (NP-SBJ-3 (PRP he)) (S_inner (VP (MD 'll) (VP (VB be) (VP (VBG looking) (S (NP-SBJ (-NONE- *-3)) (VP (TO to) (VP (VB buy) (NP (NP (NN drug) (NNS stocks)) (PP (IN like) (NP (NP (NP (NNP Eli) (NNP Lilly)) (NP_inner (, ,) (NP_inner (NP (NNP Pfizer)) (NP_inner (CC and) (NP (NNP American) (NP_inner (NNP Home) (NNP Products))))))) (SBAR (WHNP-2 (WP$ whose) (WHNP-2_inner (NN dividend) (NNS yields))) (S (NP-SBJ-1 (-NONE- *T*-2)) (VP (VBP have) (VP (VBN been) (VP (VBN bo

[ (S (NP (NNP Stock) (NNS funds)) (S (NP (VBP have) (NP (NP (NNP averaged) (NP (NP (DT a) (NP (NN staggering) (NN gain))) (PP (IN of) (NP (NP (CD 25) (NN %)) (PP (IN through) (NNP September)))))) (VP (, ,) (VP (VBG according) (PP (TO to) (NP (NNP Lipper) (NP (NNP Analytical) (NP (NNP Services) (NNP Inc))))))))) (. .)))]

GROUND TRUTH:
 (S (NP-SBJ (NN Stock) (NNS funds)) (S_inner (VP (VBP have) (VP (VBN averaged) (VP_inner (NP (NP (DT a) (NP_inner (JJ staggering) (NN gain))) (PP (IN of) (NP (CD 25) (NN %)))) (VP_inner (PP-TMP (IN through) (NP (NNP September))) (VP_inner (, ,) (PP (VBG according) (PP (TO to) (NP (NNP Lipper) (NP_inner (NNP Analytical) (NP_inner (NNP Services) (NNP Inc))))))))))) (. .)))

-------------------end of sentence-----------------

Elaine Garzarelli , who runs Shearson Lehman Hutton Inc. 's $ 335 million Sector Analysis Portfolio , predicts 0 the market will open down at least 50 points on technical factors and `` some panic selling . ''

Error: binary on insuffi

KeyboardInterrupt: 